In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from bp import data_gather_from_files,run_strategy_optimised,run_strategy_eval
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [4]:
def error_check(params,n):
    if len(params) != n:
        raise IndexError('The number of parameters is not correct')

def generate_date_ranges_for_walk_forward(start_month_year, end_month_year, day=15, n_months = 1):
    # Initialize an empty list to store the date ranges
    date_ranges = []
    if day not in range(1, 29):
        raise ValueError('Day must be between 1 and 28')
    # Convert the input strings to datetime objects, using the given day
    start_date = datetime.strptime(f"{day} {start_month_year}", '%d %b %Y')
    end_date = datetime.strptime(f"{day} {end_month_year}", '%d %b %Y')
    
    # Generate the date ranges
    current_date = start_date
    while current_date <= end_date:
        next_date = current_date + relativedelta(months=n_months)
        date_range = [current_date.strftime('%d %b %Y'), (next_date - relativedelta(days=1)).strftime('%d %b %Y')]
        date_ranges.append(date_range)
        current_date = next_date
    
    return date_ranges[:-1]

def get_previous_n_months(end_date_str, n_months):
    # Convert the input string to a datetime object
    end_date = datetime.strptime(end_date_str, '%d %b %Y')
    
    # Calculate the start date
    start_date = end_date - relativedelta(months=n_months)
    
    # Create the date range
    date_range = [start_date.strftime('%d %b %Y'), (end_date - relativedelta(days=1)).strftime('%d %b %Y')]
    
    return date_range

In [5]:
def walk_forward_analysis(evaluation_start, evaluation_end, evaluation_day,parameters,optimization_function = None, optimizer_params =[],  lookback_in_months = 6,evaluation_period = 3):
    generated_date_ranges = generate_date_ranges_for_walk_forward(evaluation_start, evaluation_end,evaluation_day,n_months = evaluation_period)
    df = {}
    for dates in generated_date_ranges:
        train_period = get_previous_n_months(dates[0], lookback_in_months)
        train_data = data_gather_from_files(train_period[0],train_period[1])['EURUSD.mid']
        print('Data gathered for training period: ',train_period[0],train_period[1])
        test_data = data_gather_from_files(dates[0],dates[1])['EURUSD.mid']
        print('Data gathered for testing period: ',dates[0],dates[1])
        max_loss, R_PNL,profit,optimal_params = optimization_function(train_data,test_data,parameters,optimizer_params)
        print('Optimal parameters are: ',optimal_params)
        #add optimization phrase evaluation
        a, b, c = run_strategy_optimised(train_data, optimal_params[0],optimal_params[1])
        print('Optimization phrase; Max loss,R_PNL,profit are: ',a, b, c)
        ##
        print('Testing phrase; Max loss, R_PNL,profit are: ',max_loss,R_PNL,profit)
        df[dates[0] +'-'+ dates[1]] = [max_loss, R_PNL,profit]
    df = pd.DataFrame(df).T
    df.columns = ['max_loss','R_PNL','profit']
    return df

In [6]:
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

In [17]:
import random
from deap import base, creator, tools

def deap_optimiser_g_n(train_data, test_data, parameters, optimization_params):
    ngen = optimization_params[0]  # number of generations
    npop = optimization_params[1]  # number of population

    error_check(parameters,2)
    
    grid_params = parameters[0]
    position_params = parameters[1]

    creator.create("FitnessMax", base.Fitness, weights=(1.0,)) #maximizing
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()

    # Define the genes for our individual
    toolbox.register("G_gene", random.randint, grid_params[0]//grid_params[2], grid_params[1]//grid_params[2])
    toolbox.register("n_gene", random.randint, position_params[0]//position_params[2], position_params[1]//position_params[2])

    # Create an individual with the genes
    toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.G_gene, toolbox.n_gene), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def objective(individual):
        G, n = individual[0]*grid_params[2], individual[1]*position_params[2]
        max_loss, R_PNL, profit = run_strategy_optimised(train_data, G, n)
    
        constraints = [
            max_loss < -500e3
        ]

        if any(constraints):
            return float('-inf'),  # Return large negative value when constraints are not satisfied
        #return profit,
        return (profit/(-(max_loss-1))),

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=[grid_params[0]//grid_params[2], position_params[0]//position_params[2]], 
                     up=[grid_params[1]//grid_params[2], position_params[1]//position_params[2]], indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)   ##############
    toolbox.register("evaluate", objective)

    population = toolbox.population(n=npop)
    CXPB, MUTPB = 0.5, 0.2

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    for gen in range(ngen):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Mutation
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        fitnesses = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit

        population[:] = offspring

    best_ind = tools.selBest(population, 1)[0]
    optimal_g = np.round(best_ind[0]*grid_params[2],5)
    optimal_n = best_ind[1]*position_params[2]
    print("optimisation completed")
    max_loss, R_PNL,profit = run_strategy_optimised(test_data, optimal_g,optimal_n)
    return max_loss, R_PNL,profit,[optimal_g,optimal_n]


In [19]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
optimizer_param = [n_trials, npop]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=3,evaluation_period=1)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Oct 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are:  [0.0015, 1500000]
Optimization phrase; Max loss,R_PNL,profit are:  -56250.0 89857.2482 79500.0
Testing phrase; Max loss, R_PNL,profit are:  -33750.0 85500.0002 85500.0
Data gathered for training period:  01 Nov 2020 31 Jan 2021
Data gathered for testing period:  01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are:  [0.0015, 1700000]
Optimization phrase; Max loss,R_PNL,profit are:  -76500.0 201619.9997 201620.0
Testing phrase; Max loss, R_PNL,profit are:  -107100.0 -80324.9999 -105910.0
Data gathered for training period:  01 Dec 2020 28 Feb 2021
Data gathered for testing period:  01 Mar 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0085, 900000]
Optimization phrase; Max loss,R_PNL,profit are

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,-33750.0,85500.0002,85500.0
01 Feb 2021-28 Feb 2021,-107100.0,-80324.9999,-105910.0
01 Mar 2021-31 Mar 2021,-64350.0,2550.0000,-64350.0
01 Apr 2021-30 Apr 2021,-154700.0,-17983.3333,-154700.0
01 May 2021-31 May 2021,-32300.0,10766.6667,6290.0
01 Jun 2021-30 Jun 2021,-120000.0,0.0000,-120000.0
01 Jul 2021-31 Jul 2021,0.0,3100.0000,3100.0
01 Aug 2021-31 Aug 2021,-45000.0,4400.0000,8800.0
01 Sep 2021-30 Sep 2021,-4080.0,1800.0000,-4080.0
01 Oct 2021-31 Oct 2021,-25200.0,64631.2500,51480.0


In [20]:
85500-105910-64350-154700+6290-120000+3100+8800-4080+51480+14200+169200

-110470